In [ ]:
import praw
from praw.models import MoreComments
import pandas as pd
import numpy as np
import time
import math
from tqdm import tqdm
from openai import OpenAI
from twikit import Client
import csv

In [ ]:
# connect to reddit api

def connect_to_reddit():
    reddit = praw.Reddit(
        client_id='FIDwbm9bpAJBbvrHNzFBVA',      
        client_secret='YCCM5UGOJKn0gP2LNViTTpZPHMzQgQ',
        user_agent='u/FecesTornado'    
    )
    return reddit

In [ ]:
# ask user for topics and source to scrape from

topic = input("what are you interested in? ")
source = int(input("which source would you like to pool from: 1 for X, 2 for reddit, 3 for both."))
numcomm = 9


In [ ]:
# strips media links and new lines from tweets

def clean_link(text):
    text = text.encode("charmap", errors="ignore").decode("charmap")
    text = text.replace('\n',' ')
    if('https' not in text): return text
    loc = text.index('https')
    return text[:loc] + text[loc+23:]

In [ ]:
if source==2:
    
    # scrape from reddit
    
    reddit=connect_to_reddit()
    
    # set up headers for csv
    data = [["text", "likes", "batch"]]
    batch = 0
    for submission in tqdm(reddit.subreddit('all').search(topic)):
        # add post and comments to data, including number of upvotes
        temp = []
        temp.append((submission.title.replace("\n", " "), submission.ups, batch))
        submission.comment_sort = "best"
        submission.comments.replace_more(limit=0)
        comments = submission.comments
        
        # ensure against index out of bounds
        for i in range(0, min(len(comments),numcomm)):
            if isinstance(comments[i], MoreComments):
                continue
            temp.append((comments[i].body.replace("\n", " "), comments[i].ups, batch))
        batch+=1
        # add post data to overall data
        data += temp

elif source == 1:
    #sign in to twitter
    USERNAME = 'YOUR_REDDIT_USERNAME'
    EMAIL = 'YOUR_REDDIT_EMAIL'
    PASSWORD = 'YOUR_REDDIT_PASSWARD'

    client = Client('en-US')

    async def main():
        await client.login(
            auth_info_1=USERNAME,
            auth_info_2=EMAIL,
            password=PASSWORD,
            cookies_file='cookies.json'
        )

    await main()


    # search for top twitter tweets after searching topic
    tweets = await client.search_tweet(topic, 'Top')
    #print(len(tweets))
    cursor = ''
    num = 500
    batch = 1
    data = [['text', 'likes', 'batch']]
    while(len(data) < num):
        for tweet in tweets:
            if(tweet.text[:5] != 'https'):
                # save post text, likes, and batch number
                data.append([f'{tweet.user.name}: {clean_link(tweet.text).strip()}', tweet.favorite_count, batch])
                #print(f"Tweeter: {tweet.user.name} \nTweet Text: {} \nLikes: {}\n")
                
                #Aggregate Replies
                tries = await client.get_tweet_by_id(tweet.id)
                replies_ = tries.replies

                #print(len(replies_))
                for rep in replies_:
                    #Save Replies into data
                    data.append([f'{tweet.user.name}: {clean_link(rep.text).strip()}', rep.favorite_count, batch])               
                    #print(f"Replier: {rep.user.name} \nReply Text: {clean_link(rep.text).strip()} \nLikes: {rep.favorite_count}\n")

                batch += 1
        #Fetches next batch of data
        tweets = await tweets.next()
else:
    #SAME AS ABOVE
    #FEWER THREADS THAN SINGULAR
    
    reddit=connect_to_reddit()
    data = [["text", "likes", "batch"]]
    batch = 0
    for submission in tqdm(reddit.subreddit('all').search(topic)):
        temp = []
        temp.append((submission.title.replace("\n", " "), submission.ups, batch))
        submission.comment_sort = "best"
        submission.comments.replace_more(limit=0)
        comments = submission.comments
        for i in range(0, min(len(comments),numcomm)):
            if isinstance(comments[i], MoreComments):
                continue
            temp.append((comments[i].body.replace("\n", " "), comments[i].ups, batch))
        batch+=1
        data += temp

    USERNAME = 'YOUR_X_USERNAME'
    EMAIL = 'YOUR_X_EMAIL'
    PASSWORD = 'YOUR_X_PASSWORD'

    client = Client('en-US')

    async def main():
        await client.login(
            auth_info_1=USERNAME,
            auth_info_2=EMAIL,
            password=PASSWORD,
            cookies_file='cookies.json'
        )

    await main()

    tweets = await client.search_tweet(topic, 'Top')
    #print(len(tweets))
    cursor = ''
    num = 250
    batch = 1
    while(len(data) < 2*num):
        for tweet in tweets:
            if(tweet.text[:5] != 'https'):
                data.append([f'{tweet.user.name}: {clean_link(tweet.text).strip()}', tweet.favorite_count, batch])
                #print(f"Tweeter: {tweet.user.name} \nTweet Text: {} \nLikes: {}\n")

                tries = await client.get_tweet_by_id(tweet.id)
                replies_ = tries.replies

                #print(len(replies_))
                for rep in replies_:
                    data.append([f'{tweet.user.name}: {clean_link(rep.text).strip()}', rep.favorite_count, batch])               
                    #print(f"Replier: {rep.user.name} \nReply Text: {clean_link(rep.text).strip()} \nLikes: {rep.favorite_count}\n")

                batch += 1

        tweets = await tweets.next()

In [ ]:
#Reak csv
with open("output.csv", 'w', newline = '', encoding="UTF-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(data)

In [ ]:
#Input the opinion you want to compare.
assessment_topic = input("What do you want to know?\n")

In [ ]:
#Input your API Key here.
apikey = "YOUR_API_KEY"

In [ ]:
#Input base url for api access
client = OpenAI(
    base_url='YOUR_BASE_URL',
    api_key=apikey
)
df = pd.read_csv("output.csv", encoding="utf-8")
user_comment = df["text"].tolist()
user_like = df["likes"].tolist()
user_batch = df["batch"].tolist()
batched_comments = []
batched_likes = []
bid = -1
for i in range(len(user_batch)):
    if i == 0 or user_batch[i] != user_batch[i-1]:
        bid += 1
        batched_comments.append([])
        batched_likes.append([])
    batched_comments[bid].append(user_comment[i])
    batched_likes[bid].append(user_like[i])

system_message = {"role": "system", "content": f"Analyze how much each user agrees with {assessment_topic}, from -100(strongly disagree) "
                                                    f"to 100(strongly agree).Response only numbers. "
                                          f"Each string in the list is a user comment, and each list is a whole thread. "
                                          f"Must generate scores for all users. Separate numbers with ; "
                                          f"Score 0 if neutral, Score -101 if irrelevant."}

In [ ]:
#Pass in all user comments, and use LLM to assess.
scores =[]
m_likes = []
for i in tqdm(range(len(batched_comments))):
    line = batched_comments[i]
    messages = [
        system_message,
        {"role": "user", "content": f"{line} ;; Response only numbers, Score 0 if neutral, Score -101 if irrelevant, separate numbers with; make sure you have all {len(line)} user ratings."}
    ]
    completion = client.chat.completions.create(
        extra_headers={},
        extra_body={},
        model="gpt-4o-2024-08-06",
        messages=messages
    )
    
    batchdata = completion.choices[0].message.content.strip().replace("\n",";").replace(".","").replace(" ", "").split(";")
    if len(batchdata) < len(batched_likes[i]):
        print(f"batch num {i} does not match")
        batched_likes[i] = batched_likes[i][:len(batchdata)]
    scores.extend(batchdata)  # Strip extra spaces/newlines
    m_likes.extend(batched_likes[i])

In [ ]:
#Preview scores
print(scores)
print(type(scores))

In [ ]:
#Summarize all data
sint = []
i_likes = []
for i in m_likes:
    try:
        i_likes.append(int(i))
    except ValueError:
        i_likes.append(0)
for x in scores:
    try:
        sint.append(int(x))
    except ValueError:
        sint.append(-101) 
sint = np.array(sint)
count_neg_10_to_neg_100 = np.sum((sint >= -100) & (sint < -10))
count_pos_10_to_100 = np.sum((sint > 10) & (sint <= 100))
count_neg_10_to_10 = np.sum((sint >= -10) & (sint <= 10))
count_irr = np.sum(sint==-101)
# 
# # Add up all scores and divide by the total number of scores (calculate the mean)
average_score = np.sum(sint[sint != -101])/np.sum(sint != -101)
# Sum all values that are not equal to -101
# # Return the results
print(f"Disagree: {count_neg_10_to_neg_100} | Agree: {count_pos_10_to_100} | Natural: {count_neg_10_to_10} | Irrelevant: {count_irr} | Average: {average_score}")


In [ ]:
#Gerenate summary of viewpoints from LLM.
messages = [
    {"role" : "system", "content" : f"You want to summarize in one paragraph the prompt {assessment_topic} base on the number we obtained. Disregard what numbers mean, only tell us the summary."},
    {"role" : "user", "content" : f"This is the data {sint}, where -101 means irrelevant, -100 to -10 is disagree, -10 to 10 is neutral, 10 to 100 is agree. {i_likes} is the like that each score has, you can correspond likes to scores for your summary."}
    ]
completion = client.chat.completions.create(
    extra_headers={},
    extra_body={},
    model="gpt-4o-2024-08-06",
    messages=messages
)
print(completion.choices[0].message.content)